In [9]:
#Algoritmo de Deutsch

from qiskit import IBMQ, BasicAer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.monitor import job_monitor
from qiskit import *
IBMQ.save_account('95d7606045a3360b89085d323d855372ccd4148396391fab24c3950349608575b7dff70964afc6803168bcedb1826fb3437c3e86658d447eccb9233795c8c3dc', overwrite=True)

qr = QuantumRegister(2)  # Initialize two qubits
cr = ClassicalRegister(2)  # Initialize two bits for record measurements
circuit = QuantumCircuit(qr, cr)

circuit.x(qr[1])  # initialize the ancilla qubit in the |1> state

circuit.barrier()

# First step of quantum algorithms - Prepare the superposition
# For superposition, we apply the Hadamard gate on both qubits
circuit.h(qr[0])
circuit.h(qr[1])

circuit.barrier()

# Oracle function
circuit.cx(qr[0], qr[1])

circuit.barrier()

# Apply Hadamard gates after querying oracle function
circuit.h(qr[0])
circuit.h(qr[1])

circuit.barrier()

# Measure qubit
circuit.measure(qr[0], cr[0])



# Run our circuit with real devices
IBMQ.load_account()
backend = provider.backends.ibmq_santiago
shots = 1024
job_exp = execute(circuit, backend=backend, shots=shots)
job_monitor(job_exp, interval=2)
results = job_exp.result()
answer = results.get_counts(circuit)
print("Real Device Result")
for c1c0 in answer:
    print(f'c0 = {c1c0[1]} ({answer[c1c0]} shots)')

ibmqfactory.load_account:WARNING:2021-05-03 14:09:40,538: Credentials are already in use. The existing account in the session will be replaced.


Job Status: job has successfully run
Real Device Result
c0 = 0 (34 shots)
c0 = 1 (990 shots)


In [10]:
# C0 ose observa como '1' en 990 shots
# Esto indica que f(0) != f(1), por lo tanto la funcion es constante. Aunque en la simulacion no se ve,
# porque no estamos comparandola con una simulacion clasica, ni midiendo tiempos, ni aumentando el numero de shots
# a un numero excesivamente grande, solo vemos reflejado que el algoritmo ha tenido exito, pero no su mejora
# respecto a un algoritmo clasico, esta mejora consiste en que desde punto de vista teorico del algoritmo de
# Deutsch, solo realizamos una consulta, mientras que en un computador clasico hariamos n consultas.
# (Como en todos los casos de backend real, hay 34 shots de error, un 3% del total)

In [47]:
#Oraculo de Deutsch-Jozsa
def dj_oracle(case, n, number):
    # We need to make a QuantumCircuit object to return
    # This circuit has n+1 qubits: the size of the input,
    # plus one output qubit
    oracle_qc = QuantumCircuit(n+1)
    
    # First, let's deal with the case in which oracle is balanced
    if case == "balanced":
        # First generate a random number that tells us which CNOTs to
        # wrap in X-gates:
        b = np.random.randint(1,2**n)
        # Next, format 'b' as a binary string of length 'n', padded with zeros:
        b_str = format(b, '0'+str(n)+'b')
        # Next, we place the first X-gates. Each digit in our binary string 
        for qubit in range(len(b_str)):
            if b_str[qubit] == number:
                oracle_qc.x(qubit)
        # Do the controlled-NOT gates for each qubit, using the output qubit 
        # as the target:
        for qubit in range(n):
            oracle_qc.cx(qubit, n)
        # Next, place the final X-gates
        for qubit in range(len(b_str)):
            if b_str[qubit] == number:
                oracle_qc.x(qubit)

    # Case in which oracle is constant
    if case == "constant":
        output = number
        if output == 1:
            oracle_qc.x(n)
    
    oracle_gate = oracle_qc.to_gate()
    oracle_gate.name = "Oracle" # To show when we display the circuit
    return oracle_gate

In [48]:
#Algoritmo de Deutsch-Jozsa
def dj_algorithm(oracle, n):
    dj_circuit = QuantumCircuit(n+1, n)
    # Set up the output qubit:
    dj_circuit.x(n)
    dj_circuit.h(n)
    # And set up the input register:
    for qubit in range(n):
        dj_circuit.h(qubit)
    # Let's append the oracle gate to our circuit:
    dj_circuit.append(oracle, range(n+1))
    # Finally, perform the H-gates again and measure:
    for qubit in range(n):
        dj_circuit.h(qubit)
    
    for i in range(n):
        dj_circuit.measure(i, i)
    
    return dj_circuit

In [57]:
#2 qubits, funcion balanceada con entrada 0
n = 2
oracle_gate = dj_oracle('balanced', n, 0)
dj_circuit = dj_algorithm(oracle_gate, n)
dj_circuit.draw()

IBMQ.load_account()
backend = provider.backends.ibmq_athens
shots = 1024
job_exp = execute(dj_circuit, backend=backend, shots=shots)
job_monitor(job_exp, interval=2)
results = job_exp.result()
answer = results.get_counts(dj_circuit)
print("Real Device Result")
for c1c0 in answer:
    print(f'c0 = {c1c0[0]} {c1c0[1]} ({answer[c1c0]} shots)')

ibmqfactory.load_account:WARNING:2021-05-03 18:42:08,055: Credentials are already in use. The existing account in the session will be replaced.


Job Status: job has successfully run
Real Device Result
c0 = 0 0 (32 shots)
c0 = 0 1 (38 shots)
c0 = 1 0 (41 shots)
c0 = 1 1 (913 shots)


In [58]:
#2 qubits, funcion balanceada con entrada 1
n = 2
oracle_gate = dj_oracle('balanced', n, 1)
dj_circuit = dj_algorithm(oracle_gate, n)
dj_circuit.draw()

backend = provider.backends.ibmq_athens
shots = 1024
job_exp = execute(dj_circuit, backend=backend, shots=shots)
job_monitor(job_exp, interval=2)
results = job_exp.result()
answer = results.get_counts(dj_circuit)
print("Real Device Result")
for c1c0 in answer:
    print(f'c0 = {c1c0[0]} {c1c0[1]} ({answer[c1c0]} shots)')

Job Status: job has successfully run
Real Device Result
c0 = 0 0 (27 shots)
c0 = 0 1 (63 shots)
c0 = 1 0 (28 shots)
c0 = 1 1 (906 shots)


In [56]:
#3 qubits, funcion balanceada con entrada 0
n = 3
oracle_gate = dj_oracle('balanced', n, 0)
dj_circuit = dj_algorithm(oracle_gate, n)
dj_circuit.draw()

backend = provider.backends.ibmq_athens
shots = 1024
job_exp = execute(dj_circuit, backend=backend, shots=shots)
job_monitor(job_exp, interval=2)
results = job_exp.result()
answer = results.get_counts(dj_circuit)
print("Real Device Result")
for c1c0 in answer:
    print(f'c0 = {c1c0[0]} {c1c0[1]} {c1c0[2]} ({answer[c1c0]} shots)')

Job Status: job has successfully run
Real Device Result
c0 = 0 0 0 (38 shots)
c0 = 0 0 1 (26 shots)
c0 = 0 1 0 (8 shots)
c0 = 0 1 1 (117 shots)
c0 = 1 0 0 (21 shots)
c0 = 1 0 1 (39 shots)
c0 = 1 1 0 (24 shots)
c0 = 1 1 1 (751 shots)


In [59]:
#3 qubits, funcion balanceada con entrada 1
n = 3
oracle_gate = dj_oracle('balanced', n, 1)
dj_circuit = dj_algorithm(oracle_gate, n)
dj_circuit.draw()

backend = provider.backends.ibmq_athens
shots = 1024
job_exp = execute(dj_circuit, backend=backend, shots=shots)
job_monitor(job_exp, interval=2)
results = job_exp.result()
answer = results.get_counts(dj_circuit)
print("Real Device Result")
for c1c0 in answer:
    print(f'c0 = {c1c0[0]} {c1c0[1]} {c1c0[2]} ({answer[c1c0]} shots)')

Job Status: job has successfully run
Real Device Result
c0 = 0 0 0 (26 shots)
c0 = 0 0 1 (27 shots)
c0 = 0 1 0 (21 shots)
c0 = 0 1 1 (128 shots)
c0 = 1 0 0 (20 shots)
c0 = 1 0 1 (41 shots)
c0 = 1 1 0 (40 shots)
c0 = 1 1 1 (721 shots)


In [60]:
#3 qubits, funcion constante con entrada 0
n = 3
oracle_gate = dj_oracle('constant', n, 0)
dj_circuit = dj_algorithm(oracle_gate, n)
dj_circuit.draw()

IBMQ.load_account()
backend = provider.backends.ibmq_athens
shots = 1024
job_exp = execute(dj_circuit, backend=backend, shots=shots)
job_monitor(job_exp, interval=2)
results = job_exp.result()
answer = results.get_counts(dj_circuit)
print("Real Device Result")
for c1c0 in answer:
    print(f'c0 = {c1c0[0]} {c1c0[1]} {c1c0[2]} ({answer[c1c0]} shots)')

ibmqfactory.load_account:WARNING:2021-05-03 19:14:10,167: Credentials are already in use. The existing account in the session will be replaced.


Job Status: job has successfully run
Real Device Result
c0 = 0 0 0 (1014 shots)
c0 = 0 0 1 (5 shots)
c0 = 0 1 0 (3 shots)
c0 = 1 0 0 (2 shots)


In [61]:
#3 qubits, funcion constante con entrada 1
n = 3
oracle_gate = dj_oracle('constant', n, 1)
dj_circuit = dj_algorithm(oracle_gate, n)
dj_circuit.draw()

IBMQ.load_account()
backend = provider.backends.ibmq_athens
shots = 1024
job_exp = execute(dj_circuit, backend=backend, shots=shots)
job_monitor(job_exp, interval=2)
results = job_exp.result()
answer = results.get_counts(dj_circuit)
print("Real Device Result")
for c1c0 in answer:
    print(f'c0 = {c1c0[0]} {c1c0[1]} {c1c0[2]} ({answer[c1c0]} shots)')

ibmqfactory.load_account:WARNING:2021-05-03 19:18:02,253: Credentials are already in use. The existing account in the session will be replaced.


Job Status: job has successfully run
Real Device Result
c0 = 0 0 0 (1011 shots)
c0 = 0 0 1 (6 shots)
c0 = 0 1 0 (1 shots)
c0 = 1 0 0 (6 shots)


In [ ]:
# Como podemos observar, en donde la funcion es constante, independientemente del valor de entrada, la salida 
# siempre es '00' en el caso de 2 qubits y '000' en el caso de 3 qubits, por lo que el resultado es correcto.
# La funcion esta parametrizada en 3 sentidos, primero en el numero de qubits, para que sea escalable, segundo 
# en la eleccion de que sea constante o balanceada y tercera en el valor de entrada para el oraculo.
# En los casos donde la salida es '11' en el caso de 2 qubits y '111' para 3 qubits significa que la funcion
# significa que la funcion es balanceada, cosa que concuerda con todos los ejemplos ejecutados anteriormente
# (Como en todos los casos de backend real, hay shots de error, siendo los de 2 qubits entorno a 5%-10% y de 3
# qubits entorno al 20%-25% de error)